In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait

import requests

import time
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.parse import unquote

import os
import cx_Oracle
import requests

# 데이터베이스 연결 설정
connection = cx_Oracle.connect('kyb/1111@localhost:1521/xe')
cursor = connection.cursor()

# 웹드라이버 설정
options = Options()
options.add_experimental_option("detach", True)
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://map.naver.com/v5/search")
time.sleep(3)  # 페이지 로딩 대기

# 검색어 목록 가져오기
cursor.execute('SELECT id, search_name FROM zero')
data_list = [(row[0], row[1]) for row in cursor.fetchall()]

# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경

# 현재 스크래핑된 HTML 요소 스냅샷 만드는 함수
def save_html_snapshot(page_str, file_prefix):
    current_date = datetime.now().strftime('%y%m%d_%H%M%S')
    file_path = f'{file_prefix}_{current_date}.html'
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(page_str)

# 해당 요소 찾을 때까지 10초 대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
        return wait
    except Exception as e:
        print(f"{code} 태그를 찾지 못하였습니다. {e}")
        driver.quit()
        return None

# 각 검색어에 대해 작업 수행
for id, search_name in data_list:
    try:
        # 검색창 찾기 및 검색어 입력
        search_box = time_wait(10, 'div.input_box > input.input_search')
        if search_box:
            search_box.clear()
            search_box.send_keys(search_name)
            search_box.send_keys(Keys.ENTER)
            time.sleep(3)  # 검색 결과 로딩 대기

            # 검색 결과 프레임으로 전환 및 스냅샷 저장
            switch_frame('searchIframe')
            save_html_snapshot(driver.page_source, f"search_result_{id}")

            # 사진 탭 클릭 및 사진 프레임으로 전환
            switch_frame('entryIframe')
            photo_tab_found = False
            for i in range(3, 6):
                try:
                    photo_tab = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, f'//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[{i}]/span'))
                    )
                    if "사진" in photo_tab.text:
                        photo_tab.click()
                        photo_tab_found = True
                        break
                except Exception as e:
                    print(f"사진 탭을 찾지 못하였습니다: {e}")
                    continue
            
            if not photo_tab_found:
                print(f"사진 탭을 찾지 못하였습니다 for id: {id}")
                continue

            time.sleep(3)  # 사진 탭 로딩 대기
            save_html_snapshot(driver.page_source, f"photo_{id}")

            # 이미지 URL 추출 및 데이터베이스 저장
            photo_urls = []
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            images = soup.find_all("img", class_="K0PDV")
            for img in images:
                image_src = img.get('src')
                if image_src:
                    decoded_url = unquote(image_src.split('&src=')[1] if '&src=' in image_src else image_src)
                    photo_urls.append(decoded_url)
                if len(photo_urls) >= 8:
                    break  # 최대 8개의 이미지 URL만 추출

            # 이미지 데이터 다운로드 및 저장
            image_data_list = []
            for url in photo_urls:
                response = requests.get(url)
                if response.status_code == 200:
                    image_data_list.append(response.content)
                else:
                    image_data_list.append(None)  # 다운로드 실패 시 None 추가

            # 데이터베이스에 이미지 저장
            cursor.execute(f"""
                INSERT INTO zero_images (id, img_num, img1, img2, img3, img4, img5, img6, img7, img8)
                VALUES (:id, Zero_img_seq.NEXTVAL, :img1, :img2, :img3, :img4, :img5, :img6, :img7, :img8)
            """, {
                'id': id,
                'img1': image_data_list[0] if len(image_data_list) > 0 else None,
                'img2': image_data_list[1] if len(image_data_list) > 1 else None,
                'img3': image_data_list[2] if len(image_data_list) > 2 else None,
                'img4': image_data_list[3] if len(image_data_list) > 3 else None,
                'img5': image_data_list[4] if len(image_data_list) > 4 else None,
                'img6': image_data_list[5] if len(image_data_list) > 5 else None,
                'img7': image_data_list[6] if len(image_data_list) > 6 else None,
                'img8': image_data_list[7] if len(image_data_list) > 7 else None
            })
            connection.commit()
    except Exception as e:
        print(f"Error occurred while processing search_name '{search_name}' with id '{id}': {e}")

# 드라이버 종료 및 연결 종료
driver.quit()
cursor.close()
connection.close()
